# Ceneo Scraper

## Struktura opinii w serwisie Ceneo.pl

|składowa|zmienna|selektor|
|--------|-------|--------|
|identyfikator|opinion_id|["data-entry-id"]|
|autora|author|span.user-post__author-name|
|rekomendację|recomendation|span.user-post__author-recomendation > em|
|liczbę gwiazdek|rating|span.user-post__score-count|
|data wystawienia opinii|opinion_date|span.user-post__published > time:nth-child(1)["datetime"]|
|data zakupu produktu|purchase_date|span.user-post__published > time:nth-child(2)["datetime"]|
|ile osób uznało opinię za przydatną|likes|button.vote-yes > span|
|ile osób uznało opinię za nieprzydatną|dislikes|buton.vote-no > span|
|treść opinii|content|div.user-post__text|
|listę wad|cons|div.review-feature__title--negatives ~ review-feature__item|
|listę zalet|pros|div.review-feature__title--positives ~ review-feature__item|

In [8]:
selectors = {
    'author': ("span.user-post__author-name",),
    'recomendation': ("span.user-post__author-recomendation > em",),
    'rating': ("span.user-post__score-count",),
    'content': ("div.user-post__text",),
    'pros': ("div.review-feature__title--positives ~ review-feature__item", None, True),
    'cons': ("div.review-feature__title--negatives ~ review-feature__item", None, True),
    'opinion_date': ("span.user-post__published > time:nth-child(1)","datetime"),
    'purchase_date': ("span.user-post__published > time:nth-child(2)","datetime"),
    'likes': ("button.vote-yes > span",),
    'dislikes': ("button.vote-no > span",),
    'opinion_id': [None, "data-entry-id"],
}

1. Import bibliotek

In [9]:
import os
import json
import requests
from bs4 import BeautifulSoup

Funckja do ekstracji zawartości ze strony HTML

In [10]:
def extract(ancestor, selector, attribute = None, return_list = False):
    if return_list:
        if attribute:
            return [tag[attribute].strip() for tag in ancestor.select(selector)]
        return [tag.get_text().strip() for tag in ancestor.select(selector)]
    if selector:
        if attribute:
            try:
                return ancestor.select_one(selector)[attribute].strip()
            except TypeError:
                return None
        try:
            return ancestor.select_one(selector).get_text().strip()
        except AttributeError:
            return None
    if attribute:
        return ancestor[attribute].strip()
    return ancestor.get_text().strip()

2. Strona produktu

In [11]:
produkt_id = "84514582"
produkt_id = input("Podaj kod produktu z Cenneo.pl")
url = f"https://www.ceneo.pl/{produkt_id}#tab=reviews"

5. Pobranie składowych pojedynczej opini

In [12]:
all_opinions = []
while(url): 
    response = requests.get(url)
    page = BeautifulSoup(response.text, "html.parser")
    opinions = page.select("div.js_product-review")
    for opinion in opinions:
        single_opinion = {
            key: extract(opinion, *value)
                for key, value in selectors.items()
        }
        all_opinions.append(single_opinion)
    try:
        url = "https://www.ceneo.pl" + extract(page, "a.pagination_next", "href")
    except TypeError:
        url = None

In [14]:
if not os.path.exists("opinios"):
    os.mkdir("opinions")
with open(f"opinions/{produkt_id}.json", "w", encoding="UTF-8") as jf:
    json.dump(all_opinions, jf, indent=4, ensure_ascii=False)